# BERT embedding

## use bert-embeddings [link](https://bert-embedding.readthedocs.io/en/latest/) (deprecated)

In [1]:
import mxnet
mxnet.__version__

'1.3.1'

Unfortunately, this only support nightly version of MXNET

## use BERT-as-a-Service （deprecated）
[https://github.com/hanxiao/bert-as-service](https://github.com/hanxiao/bert-as-service)

### download the pre-trained model
You can also manually download it [here](https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip) (all models [here](https://github.com/google-research/bert#pre-trained-models)).

In [1]:
! cd data && mkdir bert && wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-24_H-1024_A-16.zip

'wget' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


### server side
- install the package

In [8]:
!pip install bert-serving-server tensorflow

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


- start the service

In [9]:
# !bert-serving-start -model_dir ./data/bert/uncased_L-24_H-1024_A-16 -num_worker=1
# do not run this in jupyter notebook, run in shell instead

2019-05-08 20:21:18.036060: F tensorflow/python/lib/core/bfloat16.cc:675] Check failed: PyBfloat16_Type.tp_base != nullptr 


### client side

In [3]:
!pip install bert-serving-client

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [ ]:
from bert_serving.client import BertClient
bc = BertClient()
bc.encode(['First do it', 'then do it right', 'then do it better'])